# **Variability of Young Stellar Objects — Internship Report**  
**Author:** Chiara Virzì  
**Date:** April 2025

This report outlines the work I conducted during my internship under the supervision of Dr. Rosaria Bonito at the Osservatorio Astronomico di Palermo. The focus of the internship was the analysis of variability in Young Stellar Objects (YSOs), using both photometric and spectroscopic data, in preparation for the upcoming Rubin LSST survey.

# **1. Introduction**  
The aim of this project is to study spectral variability in Young Stellar Objects (YSOs), with a focus on the Hα emission line, across timescales of several months. As a Rubin LSST data rights holder, I developed methods to identify accretion signatures in archival spectra and correlated these features with photometric variability. The Hα line serves as a key diagnostic for accretion processes, mass-loss activity, and the evolutionary stages of pre-main-sequence stars. 

# **2. Young Stellar Object Science**  
YSOs are stars in their early formation stages, still undergoing contraction and typically surrounded by disks of gas and dust. These stars are known for their strong variability, driven by accretion, magnetic activity, and interactions with surrounding circumstellar material.

### **YSO Evolutionary Classes**  
YSOs are classified based on their spectral energy distributions (SEDs) and infrared excess:

- **Class 0**: Deeply embedded protostars, observed mainly in the submillimeter range.  
- **Class I**: Protostars with infalling envelopes, showing strong infrared excess and outflows.  
- **Class II**: Classical T Tauri Stars (CTTS) with developed accretion disks and active mass transfer.  
- **Class III**: Weak-lined T Tauri Stars (WTTS) with little or no disk and minimal accretion signatures.  

Understanding Class II and III sources is crucial for studying late-stage star formation and early disk evolution—key elements for understanding planet formation. [4](#ref4)

## **Accretion Bursts**  
Accretion bursts are episodic events in which disk material rapidly accretes onto the stellar surface, temporarily increasing brightness and temperature. Localized hot spots dominate the UV/blue spectrum during these events.  

Different photometric bands respond uniquely:

- **u/g bands**: Trace hot-spot and shock emission.  
- **i/z bands**: Sensitive to extinction changes and disk occultation.

### **Short-Term Variability**  
Occurs over hours to days, driven by magnetic flares, unstable accretion streams, or jet activity.

### **Long-Term Variability**  
Occurs over months to years, including EXor-type (months-long) and FUor-type (decades-long) outbursts.In particular EXors, characterized by episodic accretion outbursts lasting months to years, are still poorly understood due to the rarity of available samples. 

## **Light Curves**  
Light curves record brightness vs. time, revealing:

- **Periodic variability** from rotation or disk warps.  
- **Aperiodic variability** from changing accretion or extinction.

In this project, ZTF light curves were cross-matched with Hα spectroscopic diagnostics to identify correlated variability. [12](#ref12)

## **Spectroscopy and the Hα Line**  
The Hα line at 6562.8 Å provides direct insight into accretion and wind phenomena.

- **Broad/asymmetric profiles** indicate magnetospheric infall or outflow.  
- **Narrow/symmetric profiles** indicate chromospheric emission.  

The Full Width at Zero Intensity (FWZI) measures the total emission-line width—from continuum departure to return—and serves as a proxy for maximum gas velocities in the line-forming region.

# **3. Rubin LSST and 4MOST**  

The **Rubin Observatory’s LSST** (Legacy Survey of Space and Time) will image the sky in six bands (*u, g, r, i, z, y*) over ten years, enabling both short- and long-term variability studies.  

<p align="center">
  <img src="./immaginireport/filters_lsst.jpeg" width="500"><br>
  <em><small>Fig 1: LSST filter transmission curves, including atmospheric and telescope throughput.</small></em>
</p>

The **4MOST** facility will provide time-resolved spectroscopy for variable-source follow-up. In this study, the NGC 2264 region—also observed by Gaia-ESO—serves as a testbed for the Rubin–4MOST synergy.

### **Brokers**  
Real-time brokers (ANTARES, ALeRCE) filter and classify survey alerts, providing light curves, forced photometry, and ML-based object classification to facilitate follow-up observations.

# **4. Methods**

## **Temporal Study of Variability**  
We implemented a pipeline that:

1. Selects alerts in ANTARES and favorites the target.  
2. Retrieves light curves and classifications from ALeRCE.  
3. Analyzes photometric variability patterns.  
4. Compares archival Gaia-ESO spectra (Hα profiles) across epochs.

<p align="center">
  <img src="immaginireport/LC.png" width="500"><br>
  <em><small>Fig 2: ALeRCE light curve showing forced photometry and non-detections.</small></em>
</p>




Below are the libraries used and the core function for FITS data reading:


In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [14]:
# Function to read FITS files
def read_fits(file):
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        data = hdul[0].data

    crval1 = header['CRVAL1']
    crpix1 = header['CRPIX1']
    cdelt1 = header['CDELT1']
    npix   = header['NAXIS1']

    # Wavelenght (in Å)
    x = crval1 + (np.arange(npix) - (crpix1 - 1)) * cdelt1
    return x, data
    

## **Spectroscopic Analysis and Sky Contribution**

To verify the variability of the object, I examined the raw spectroscopic data, focusing on the Hα line region. I selected six spectra spanning approximately three months that exhibit pronounced changes in both line intensity and profile shape.

<p align="center">
  <img src="immaginireport/all_spectra.png" width="500">
  <br>
  <em><small>Fig 3: Example of spectral variability in the Hα line. Variations in both intensity and profile shape are visible across different epochs.</small></em>
</p>

The analysis reveals significant variability in the Hα profile, both in intensity and shape, confirming the presence of dynamic physical processes such as variable mass accretion. This behavior was previously reported by [Bonito et al. (2020)](#bonito2020), and is consistent with known variability patterns in young stellar objects.  
The spectra used in this step were pre-normalized and pre-trimmed by Dr. Bonito.


Next, I performed a quantitative comparison between sky-subtracted and non-sky-subtracted spectra from the Gaia-ESO Survey (Data Release 4) for NGC 2264. This comparison assesses how the sky-subtraction procedure affects spectral diagnostics, particularly in regions of strong nebular emission.


[Bonito et al. (2020)](#bonito2020) proposed a classification scheme based on the FWZI, of the Hα line:

- **Good cases (confident accretors)**:  
  - FWZI(Hα) > 14 Å  
  - No artificial absorption from over-subtraction  
  - Clear emission profile  

- **Bad cases (non-accretors)**:  
  - FWZI(Hα) < 3 Å  
  - Profile dominated by sky-subtraction residuals  
  - Measurements unreliable  

This study begins with analysis of a **bad case** and its sky-subtracted spectrum.

<p align="center">
  <img src="immaginireport/bad_skysub.png" width="500">
  <br>
  <em><small>Fig 5: Sky-subtracted spectrum classified as a "bad case." Over-subtraction produces spurious absorption features.</small></em>
</p>

The sky-subtracted spectrum for this non-accreting object shows over-subtraction artifacts, manifesting as spurious absorption features in the Hα region. These distortions make any FWZI measurement unreliable, so it was decided to analize the non-sky-subtracted spectrum.

But before that to ensure that observed Hα variability is intrinsic to the star rather than background contamination, I analyzed ten pure-sky spectra. For each, I isolated the Hα region and measured the wavelengths at which nebular emission rises and falls. The average values are:

<p align="center">
  <img src="immaginireport/sky.png" width="500">
  <br>
  <em><small>Fig 4: Example of a sky spectrum around the Hα region.</small></em>
</p>

- **Mean λ min**: 6562.20 Å  
- **Mean λ max**: 6563.90 Å  
- **Mean nebular FWZI**: 1.70 Å  

<p align="center">
  <img src="immaginireport/bad_noskysub.png" width="500">
  <br>
  <em><small>Fig 6: Non-sky-subtracted spectrum. Nebular emission dominates, obscuring the stellar line.</small></em>
</p>

Extracted parameters:

- **λ min**: 6561.95 Å  
- **λ max**: 6564.30 Å  
- **FWZI**: 2.35 Å  

Since FWZI < 3 Å, this object is classified as a non-accretor.

### **Confirmed Accretor (Good Case)**

A second object, previously identified as a Class II accretor by [Venuti et al. (2018)](#venuti2018), was analyzed to validate our methodology. Both sky-subtracted and raw spectra show nearly identical Hα profiles, indicating minimal nebular contamination.

<p align="center">
  <img src="immaginireport/best_noskysub.png" width="500">
  <br>
  <em><small>Fig 7: Sky-subtracted (pink) vs. non-sky-subtracted (black) spectra for a confirmed accretor. Profiles overlap within noise.</small></em>
</p>

<p align="center">
  <img src="immaginireport/best_line.png" width="500">
  <br>
  <em><small>Fig 8: Raw spectrum with identified λ min and λ max for FWZI measurement.</small></em>
</p>
Given the quality of the data, I proceeded with a more quantitative analysis, including the measurement of:
- Hα at 10% of the peak (Hα 10%)   
- Radial velocities from the line wings:v_blue (Maximum blueshifted velocity where emission is still present. A negative velocity indicates material moving toward us, often interpreted as infalling accreting gas or stellar winds ) and v_red (Maximum redshifted velocity, where emission extends on the red wing of the line. This is material moving away from us, potentially indicating infall along the opposite side of the star, or disk rotation)     
- Mass accretion rate

Derived metrics:
- **FWZI** = 19.95 Å  
- **Hα 10% width** = 578.26 km/s  
- **v_blue** = –446 km/s; **v_red** = +466 km/s  
- **log Ṁ_acc** (Venuti 2014) = –7.28  

### **Mass Accretion Rate Estimation**
To estimate the mass accretion rate (Ṁ), i followed the method outlined by [Natta et al. (2004)](#natta2004), which originally targeted brown dwarfs. However, since that relation may not be perfectly adapted to our dataset, i recalibrated the empirical constants by applying the same logic to a sample of confirmed accretors ("best cases") extracted from the VizieR catalog associated with [Bonito et al. (2020)](#bonito2020).

I plotted the logarithm of the mass accretion rate from Natta’s study against the FWZI of Hα for our selected sources. The linear fit yielded the following relationship:

log(Ṁ_acc) = 0.406 × FWZI – 14.823

<p align="center">
  <img src="immaginireport/natta.png" width="500">
  <br>
  <em><small>Fig 9: Regression of log Ṁ_acc vs. Hα FWZI for calibration sample.</small></em>
</p>
Using this calibration, i computed a new accretion rate for the object under study, obtaining:

- **log Ṁ_acc (custom)** = –6.82  
This result is consistent with the values reported in the VizieR catalog, validating the method.  

## **5. Future Perspectives**
This report aims to highlight the potential of future surveys, particularly the **Rubin LSST**, for the study of YSOs. Rubin will act as a **discovery machine** revealing long-term variable objects such as EXors.As an example, I explored the case of a newly reported EXor-like object and assessed its detectability using LSST-like cadence and filters.
From the broker Alerce

<p align="center">
  <img src="immaginireport/LC_exori.png" width="500">
  <br>
  <em><small>Fig 10: light curve of an EXor, from the broker ALeRCE.</small></em>
</p>

The Rubin LSST will revolutionize YSO studies by:

- Discovering large populations of EXor-type outbursts (months-long events)  
- Enabling statistical analyses of episodic accretion across clusters  
- Facilitating prompt spectroscopic follow-up via broker alerts and 4MOST  

## **6. Discussion and Conclusion**

This report presents a simulation of the future Rubin LSST–4MOST synergy, focusing on the analysis of young stellar objects (YSOs) in the NGC 2264 region. I investigated both photometric and spectroscopic variability, using the Hα emission line and nearby forbidden lines as diagnostic tools.

It is possible to verify the variability of a YSO by monitoring how its Hα line evolves over time. Moving toward a more quantitative analysis, I examined the spectra of two distinct objects. The first object represents a bad case: in particular, its sky-subtracted spectrum exhibited absorption features likely caused by over-subtraction, while the corresponding non-sky-subtracted spectrum was also unsuitable for further analysis due to dominant nebular noise overpowering the stellar signal.

This type of analysis enables rapid classification of cluster members, significantly reducing the volume of data that requires deeper examination.

In contrast, the second object—deliberately selected as a good case—presents two nearly identical spectra. Furthermore, the region most affected by nebular contamination does not interfere with our diagnostic line, thereby confirming the reliability of the data. Proceeding with a more detailed analysis, I calculated several physical parameters of interest.

In particular, I focused on the mass accretion rate, adapting existing calibrations to our specific case using data collected from the VizieR catalog of Bonito et al. (2020). Using FWZI as a proxy for accretion, we developed a custom calibration for the mass accretion rate and applied it to well-classified sources, obtaining consistent and meaningful results.

Although this methodology was demonstrated using only two objects, it represents a potentially powerful tool for statistically identifying accretors across large stellar populations.


## **7. Cross-Disciplinary Skills Acquired**

- Handling and analysis of astronomical data 
- Use of Python for scientific programming and data visualization
- Working with data brokers (ALeRCE, ANTARES)
- Basic use of GitHub 
- Scientific writing and presentation of results
- Had a presentation for the Rubin science Platform abpout brokers
- Attendance to the zoom meeting on RSP platform


## References

   
- <a name="ref11"></a>Overview of YSO variability mechanisms.  
- <a name="ref12"></a>ZTF public data release and light-curve applications.  
- <a name="ref13"></a>Definition and use of FWZI in YSO accretion studies.  
- <a name="ref14"></a>Street, R., et al. (2023). *Rubin–4MOST synergy in NGC 2264*.  
- <a name="bonito2020"></a>Bonito, R., et al. (2020). *Accretion diagnostics in NGC 2264 from the Gaia-ESO Survey*. Astronomy & Astrophysics, 644, A62. [ADS](https://ui.adsabs.harvard.edu/abs/2020A%26A...644A..62B)  
- <a name="venuti2018"></a>Venuti, L., et al. (2018). *Accretion and rotation in NGC 2264: a multi-wavelength study*. Astronomy & Astrophysics, 609, A10. [ADS](https://ui.adsabs.harvard.edu/abs/2018A%26A...609A..10V)  
- <a name="natta2004"></a>Natta, A., et al. (2004). *Accretion in young brown dwarfs: empirical relations*. Astronomy & Astrophysics, 424, 603. [ADS](https://ui.adsabs.harvard.edu/abs/2004A%26A...424..603N)